In [30]:
# importing and configering libs

In [31]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',2000) # show all columns in jupyter
pd.set_option('display.max_rows', 100) # display 100 at least rows

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [32]:
# Loading the data

In [33]:
colsToLoad =["INSTNM","ADM_RATE","ADM_RATE_ALL","ACTCMMID","ACTENMID","ACTMTMID","ACTWRMID","SAT_AVG","SAT_AVG_ALL","SATMTMID","UGDS","HIGHDEG", "CONTROL", "TUITFTE", "INEXPFTE", "AVGFACSAL", "COSTT4_P",
             "COSTT4_A", "PCTFLOAN", "PCTPELL", "COMP_ORIG_YR2_RT","UGDS_WHITE","UGDS_BLACK","UGDS_HISP","UGDS_ASIAN","UGDS_AIAN","UGDS_NHPI","UGDS_2MOR","UGDS_NRA","UGDS_UNKN","PPTUG_EF","COSTT4_A","COSTT4_P","TUITIONFEE_IN","TUITIONFEE_OUT","TUITIONFEE_PROG","TUITFTE","INEXPFTE","AVGFACSAL","PCTPELL","DEATH_YR3_RT","COMP_ORIG_YR3_RT","LOAN_DEATH_YR3_RT","LOAN_COMP_ORIG_YR3_RT","DEATH_YR4_RT","COMP_ORIG_YR4_RT","COMPL_RPY_1YR_RT","NONCOM_RPY_1YR_RT","AGE_ENTRY","COUNT_NWNE_P10","COUNT_WNE_P10","MN_EARN_WNE_P10","MD_EARN_WNE_P10","COMPL_RPY_1YR_RT","NONCOM_RPY_1YR_RT","COMPL_RPY_3YR_RT","NONCOM_RPY_3YR_RT","COMPL_RPY_5YR_RT","NONCOM_RPY_5YR_RT","COMPL_RPY_7YR_RT","NONCOM_RPY_7YR_RT"]
scoreCardDF = pd.read_csv("I:\\NU Project\\Git-Repo\\Data\\MERGED2013_14_PP.csv", index_col="INSTNM",usecols=colsToLoad)

# ignore "MEDIAN_HH_INC" column, as 0 non-null float64

F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1606) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
# General exloration of data 

In [35]:
scoreCardDF.shape

(7804, 52)

In [36]:
scoreCardDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7804 entries, Alabama A & M University to Georgia Military College-Stone Mountain
Data columns (total 52 columns):
HIGHDEG                  7804 non-null int64
CONTROL                  7804 non-null int64
ADM_RATE                 2221 non-null float64
ADM_RATE_ALL             2486 non-null float64
SATMTMID                 1317 non-null float64
ACTCMMID                 1344 non-null float64
ACTENMID                 1167 non-null float64
ACTMTMID                 1168 non-null float64
ACTWRMID                 300 non-null float64
SAT_AVG                  1422 non-null float64
SAT_AVG_ALL              1532 non-null float64
UGDS                     7091 non-null float64
UGDS_WHITE               7091 non-null float64
UGDS_BLACK               7091 non-null float64
UGDS_HISP                7091 non-null float64
UGDS_ASIAN               7091 non-null float64
UGDS_AIAN                7091 non-null float64
UGDS_NHPI                7091 non-null float64

In [37]:
scoreCardDF.head()

,HIGHDEG,CONTROL,ADM_RATE,ADM_RATE_ALL,SATMTMID,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SAT_AVG,SAT_AVG_ALL,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PCTPELL,PCTFLOAN,COMP_ORIG_YR2_RT,DEATH_YR3_RT,COMP_ORIG_YR3_RT,LOAN_DEATH_YR3_RT,LOAN_COMP_ORIG_YR3_RT,DEATH_YR4_RT,COMP_ORIG_YR4_RT,COMPL_RPY_1YR_RT,NONCOM_RPY_1YR_RT,COMPL_RPY_3YR_RT,NONCOM_RPY_3YR_RT,COMPL_RPY_5YR_RT,NONCOM_RPY_5YR_RT,COMPL_RPY_7YR_RT,NONCOM_RPY_7YR_RT,AGE_ENTRY,COUNT_NWNE_P10,COUNT_WNE_P10,MN_EARN_WNE_P10,MD_EARN_WNE_P10
INSTNM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alabama A & M University,4,1,0.8989,0.8989,400.0,17.0,17.0,17.0,NaN,823.0,823.0,4051.0,0.0279,0.9501,0.0089,0.0022,0.0012,0.0010,0.0000,0.0002,0.0084,0.0622,18888.0,NaN,7182.0,12774.0,NaN,9063.0,7459.0,7079.0,0.7115,0.8204,0.034958602,PrivacySuppressed,0.097451274,PrivacySuppressed,0.087332809,PrivacySuppressed,0.241845664,0.3075801749,0.1213692946,0.388185654,0.1768901569,0.5305514158,0.2299150882,0.7553324969,0.4143646409,20.540410133,246,1741,34300,29900
University of Alabama at Birmingham,4,1,0.8673,0.8673,585.0,25.0,26.0,23.0,NaN,1146.0,1146.0,11200.0,0.5987,0.2590,0.0258,0.0518,0.0026,0.0007,0.0344,0.0140,0.0130,0.2579,19990.0,NaN,7206.0,16398.0,NaN,9033.0,17208.0,10170.0,0.3505,0.5397,0.150529204,PrivacySuppressed,0.333945628,PrivacySuppressed,0.332664527,PrivacySuppressed,0.422933112,0.578436134,0.3365384615,0.6351660939,0.4240231548,0.7312626604,0.5116761806,0.8288419519,0.6054718477,23.45881476,470,3474,46400,40200
Amridge University,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,322.0,0.2919,0.4224,0.0093,0.0031,0.0031,0.0031,0.0000,0.0000,0.2671,0.3727,12300.0,NaN,6870.0,6870.0,NaN,12057.0,5123.0,3849.0,0.6839,0.7629,0.058823529,PrivacySuppressed,0.161849711,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,0.209580838,0.3859649123,0.1542699725,0.6226415094,0.3043478261,0.5192307692,0.4144144144,PrivacySuppressed,PrivacySuppressed,33.925581395,57,168,46100,40100
University of Alabama in Huntsville,4,1,0.8062,0.8062,580.0,26.0,26.0,25.0,NaN,1180.0,1180.0,5525.0,0.7012,0.1310,0.0338,0.0364,0.0145,0.0002,0.0161,0.0329,0.0338,0.2395,20306.0,NaN,9192.0,21506.0,NaN,8322.0,9352.0,9341.0,0.3281,0.4728,0.157939189,PrivacySuppressed,0.284602076,PrivacySuppressed,0.283109405,PrivacySuppressed,0.393046107,0.639589169,0.3486956522,0.7177142857,0.3818860878,0.7959731544,0.474012474,0.8810810811,0.6389140271,24.10127697,188,1361,50500,45600
Alabama State University,4,1,0.5125,0.5125,425.0,17.0,17.0,17.0,NaN,830.0,830.0,5354.0,0.0161,0.9285,0.0114,0.0015,0.0009,0.0007,0.0064,0.0207,0.0138,0.0902,17400.0,NaN,8720.0,15656.0,NaN,7813.0,7393.0,6557.0,0.8265,0.8735,0.028949025,PrivacySuppressed,0.115574348,PrivacySuppressed,0.10949464,PrivacySuppressed,0.187935035,0.2118863049,0.0985550204,0.3057553957,0.1694465504,0.4466019417,0.2263289037,0.7058823529,0.3645320197,20.648575305,285,1977,29500,26700


In [38]:
scoreCardDF.describe()

,HIGHDEG,CONTROL,ADM_RATE,ADM_RATE_ALL,SATMTMID,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SAT_AVG,SAT_AVG_ALL,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PCTPELL,PCTFLOAN
count,7804.000000,7804.000000,2221.000000,2486.000000,1317.000000,1344.000000,1167.000000,1168.000000,300.000000,1422.000000,1532.000000,7091.000000,7091.000000,7091.000000,7091.000000,7091.000000,7091.000000,7091.000000,7091.000000,7091.000000,7091.000000,7072.000000,4139.000000,2543.000000,4506.000000,4287.000000,2748.000000,7362.000000,7362.000000,4654.000000,7063.000000,7063.000000
mean,2.180420,2.216427,0.683853,0.692173,530.869400,23.125744,22.730077,22.590753,7.736667,1056.829817,1061.605091,2332.675363,0.517640,0.187379,0.156696,0.032716,0.013823,0.004473,0.022181,0.014683,0.049281,0.224013,24355.081662,18481.605977,13857.479361,16305.035223,14910.383916,9962.513447,6934.007335,6014.295875,0.528684,0.531370
std,1.317085,0.837223,0.208593,0.202677,71.719976,3.421468,3.764050,3.395695,1.054052,131.817989,129.958729,5444.538996,0.287867,0.222103,0.220069,0.073006,0.068906,0.031648,0.032064,0.045770,0.101418,0.245676,12170.180307,6358.082813,10523.846885,9790.016165,7127.349341,8404.164486,8850.919248,2264.836833,0.224824,0.285514
min,0.000000,1.000000,0.000000,0.000000,310.000000,2.000000,2.000000,2.000000,5.000000,666.000000,666.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4157.000000,3057.000000,0.000000,80.000000,263.000000,0.000000,0.000000,269.000000,0.000000,0.000000
25%,1.000000,1.000000,0.554300,0.563350,483.000000,21.000000,20.000000,21.000000,7.000000,972.000000,980.000000,121.000000,0.278950,0.035100,0.025600,0.002300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14151.500000,14164.500000,5293.000000,8918.000000,11199.500000,4365.500000,3275.250000,4471.250000,0.358300,0.343550
50%,2.000000,2.000000,0.701600,0.718500,520.000000,23.000000,22.000000,22.000000,7.000000,1037.500000,1047.500000,431.000000,0.565000,0.099300,0.068500,0.011900,0.002700,0.000000,0.015500,0.000000,0.014600,0.150350,22865.000000,17786.000000,12094.000000,14257.000000,14600.000000,8758.000000,5193.000000,5794.000000,0.516500,0.592600
75%,4.000000,3.000000,0.835000,0.838784,565.000000,25.000000,25.000000,24.000000,9.000000,1117.750000,1122.000000,1922.000000,0.759600,0.254550,0.187750,0.031150,0.007400,0.002400,0.030900,0.010150,0.048250,0.372700,30372.500000,22590.500000,18048.000000,20450.000000,17000.000000,13411.000000,7852.000000,7264.750000,0.704550,0.757600
max,4.000000,3.000000,1.000000,1.000000,785.000000,34.000000,34.000000,35.000000,12.000000,1534.000000,1534.000000,166816.000000,1.000000,1.000000,1.000000,0.971800,1.000000,1.000000,1.000000,0.902400,1.000000,1.000000,74473.000000,89422.000000,64900.000000,64900.000000,88550.000000,207213.000000,307089.000000,23611.000000,1.000000,1.000000


In [39]:
def cleanPrivacySuppressed(dataFrame):
    for colName in dataFrame.columns:
        dataFrame[colName] = dataFrame[colName].apply(lambda x: np.nan if x =="PrivacySuppressed" else x)
        dataFrame[colName]= pd.to_numeric(dataFrame[colName])

In [40]:
#some columns have PrivacySuppressed as a value to indicate that the value is missing, but it is a column of type float
#let's clean that
cleanPrivacySuppressed(scoreCardDF)
scoreCardDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7804 entries, Alabama A & M University to Georgia Military College-Stone Mountain
Data columns (total 52 columns):
HIGHDEG                  7804 non-null int64
CONTROL                  7804 non-null int64
ADM_RATE                 2221 non-null float64
ADM_RATE_ALL             2486 non-null float64
SATMTMID                 1317 non-null float64
ACTCMMID                 1344 non-null float64
ACTENMID                 1167 non-null float64
ACTMTMID                 1168 non-null float64
ACTWRMID                 300 non-null float64
SAT_AVG                  1422 non-null float64
SAT_AVG_ALL              1532 non-null float64
UGDS                     7091 non-null float64
UGDS_WHITE               7091 non-null float64
UGDS_BLACK               7091 non-null float64
UGDS_HISP                7091 non-null float64
UGDS_ASIAN               7091 non-null float64
UGDS_AIAN                7091 non-null float64
UGDS_NHPI                7091 non-null float64

In [41]:
# Assuming that the output or target variable is COMPL_RPY_1YR_RT

In [42]:
# get the number of null values at each column
scoreCardDF.isnull().sum()

HIGHDEG                     0
CONTROL                     0
ADM_RATE                 5583
ADM_RATE_ALL             5318
SATMTMID                 6487
ACTCMMID                 6460
ACTENMID                 6637
ACTMTMID                 6636
ACTWRMID                 7504
SAT_AVG                  6382
SAT_AVG_ALL              6272
UGDS                      713
UGDS_WHITE                713
UGDS_BLACK                713
UGDS_HISP                 713
UGDS_ASIAN                713
UGDS_AIAN                 713
UGDS_NHPI                 713
UGDS_2MOR                 713
UGDS_NRA                  713
UGDS_UNKN                 713
PPTUG_EF                  732
COSTT4_A                 3665
COSTT4_P                 5261
TUITIONFEE_IN            3298
TUITIONFEE_OUT           3517
TUITIONFEE_PROG          5056
TUITFTE                   442
INEXPFTE                  442
AVGFACSAL                3150
PCTPELL                   741
PCTFLOAN                  741
COMP_ORIG_YR2_RT         1195
DEATH_YR3_

In [43]:
# replace null values with mean value across all columns of dataframe
def replaceNullWithMean(dataFrame , replace):
    for colName in dataFrame.columns:
        column_mean = dataFrame[colName].mean()
        dataFrame[colName].fillna(column_mean, inplace=replace)

In [45]:
# imputation using mean value
replaceNullWithMean(scoreCardDF, True)
scoreCardDF.isnull().sum()

HIGHDEG                  0
CONTROL                  0
ADM_RATE                 0
ADM_RATE_ALL             0
SATMTMID                 0
ACTCMMID                 0
ACTENMID                 0
ACTMTMID                 0
ACTWRMID                 0
SAT_AVG                  0
SAT_AVG_ALL              0
UGDS                     0
UGDS_WHITE               0
UGDS_BLACK               0
UGDS_HISP                0
UGDS_ASIAN               0
UGDS_AIAN                0
UGDS_NHPI                0
UGDS_2MOR                0
UGDS_NRA                 0
UGDS_UNKN                0
PPTUG_EF                 0
COSTT4_A                 0
COSTT4_P                 0
TUITIONFEE_IN            0
TUITIONFEE_OUT           0
TUITIONFEE_PROG          0
TUITFTE                  0
INEXPFTE                 0
AVGFACSAL                0
PCTPELL                  0
PCTFLOAN                 0
COMP_ORIG_YR2_RT         0
DEATH_YR3_RT             0
COMP_ORIG_YR3_RT         0
LOAN_DEATH_YR3_RT        0
LOAN_COMP_ORIG_YR3_RT    0
D

In [48]:
# Preparing data for regression model
# X >> represents input features , Y >> represents the output or target feature that should b predicted using the regression model

X = scoreCardDF[['ADM_RATE','ADM_RATE_ALL','ACTCMMID','ACTENMID','ACTMTMID','ACTWRMID','SAT_AVG','SAT_AVG_ALL','SATMTMID','UGDS','HIGHDEG', 'CONTROL', 'TUITFTE', 'INEXPFTE', 'AVGFACSAL', 'COSTT4_P',
             'COSTT4_A', 'PCTFLOAN', 'PCTPELL', 'COMP_ORIG_YR2_RT','UGDS_WHITE','UGDS_BLACK','UGDS_HISP','UGDS_ASIAN','UGDS_AIAN','UGDS_NHPI','UGDS_2MOR','UGDS_NRA','UGDS_UNKN','PPTUG_EF','COSTT4_A','COSTT4_P','TUITIONFEE_IN','TUITIONFEE_OUT','TUITIONFEE_PROG','TUITFTE','INEXPFTE','AVGFACSAL','PCTPELL','DEATH_YR3_RT','COMP_ORIG_YR3_RT','LOAN_DEATH_YR3_RT','LOAN_COMP_ORIG_YR3_RT','DEATH_YR4_RT','COMP_ORIG_YR4_RT','AGE_ENTRY','COUNT_NWNE_P10','COUNT_WNE_P10','MN_EARN_WNE_P10','MD_EARN_WNE_P10']]
Y = scoreCardDF['COMPL_RPY_1YR_RT']

In [49]:
# Divide the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [51]:
# Training the algorithm (Multiple Linear Regression)
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [52]:
#  in case of multivariable linear regression, the regression model has to find the most optimal coefficients for all the attributes. 
# To see what coefficients our regression model has chosen
coeffDF = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeffDF

,Coefficient
ADM_RATE,7.178832e-02
ADM_RATE_ALL,-7.136196e-02
ACTCMMID,-2.549119e-03
ACTENMID,2.343294e-03
ACTMTMID,-3.174089e-03
ACTWRMID,-1.214154e-02
SAT_AVG,4.101995e-05
SAT_AVG_ALL,2.485670e-04
SATMTMID,-2.337472e-04
UGDS,-8.628832e-07


In [53]:
# To make predictions on the test data
Y_pred = regressor.predict(X_test)
# To compare the actual output values for X_test which is (Y_test) with the predicted values Y_pred
df = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
df

,Actual,Predicted
INSTNM,,
Wilberforce University,0.401709,0.383830
Sierra College of Beauty,0.488447,0.474094
Robert Fiance Beauty Schools-North Plainfield,0.488447,0.402466
Washington University in St Louis,0.890519,0.839849
ITT Technical Institute-Lake Mary,0.282946,0.354089
...,...,...
Gerbers Akron Beauty School,0.488447,0.446814
Presentation College,0.771429,0.552590
Norwalk Community College,0.488447,0.546776


In [54]:
# The final step is to evaluate the performance of algorithm. 
# We'll do this by finding the values for MAE, MSE and RMSE
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, Y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

Mean Absolute Error: 0.08320071708317381
Mean Squared Error: 0.011608941018964352
Root Mean Squared Error: 0.10774479578598845


In [56]:
targetFeatureMean = scoreCardDF['COMPL_RPY_1YR_RT'].mean()
targetFeatureMean

0.4884474203218558